In [ ]:
import sqlite3
from datetime import datetime, timedelta
import time

# In-memory database setup
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create tables
cursor.executescript('''
CREATE TABLE Authors (
    author_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    bio TEXT
);

CREATE TABLE Genres (
    genre_id INTEGER PRIMARY KEY,
    genre_type TEXT,
    description TEXT
);

CREATE TABLE Books (
    book_id INTEGER PRIMARY KEY,
    title TEXT,
    published_date TEXT,
    price REAL,
    author_id INTEGER,
    genre_id INTEGER,
    FOREIGN KEY (author_id) REFERENCES Authors(author_id),
    FOREIGN KEY (genre_id) REFERENCES Genres(genre_id)
);

CREATE TABLE Reviews (
    review_id INTEGER PRIMARY KEY,
    book_id INTEGER,
    rating INTEGER,
    review_text TEXT,
    review_date TEXT,
    FOREIGN KEY (book_id) REFERENCES Books(book_id)
);
''')

# Caching mechanism
cache = {}

def cache_search(query, result):
    cache[query] = {
        'result': result,
        'timestamp': datetime.now()
    }

def get_from_cache(query):
    if query in cache:
        if datetime.now() - cache[query]['timestamp'] < timedelta(minutes=30):  # 30 minutes cache duration
            return cache[query]['result']
        else:
            del cache[query]  # Remove expired cache
    return None

# Session management
sessions = {}

def create_session(user_id):
    sessions[user_id] = {
        'start_time': datetime.now()
    }

def is_session_valid(user_id):
    if user_id in sessions:
        if datetime.now() - sessions[user_id]['start_time'] < timedelta(hours=1):  # 1 hour session duration
            return True
        else:
            del sessions[user_id]  # Session expired
    return False

# Mock data insertion
authors = [
    ('John', 'Doe', 'An author'),
    ('Jane', 'Smith', 'Another author'),
    ('Alice', 'Johnson', 'Yet another author')
]

genres = [
    ('Fiction', 'Fictional books'),
    ('Science Fiction', 'Sci-Fi books'),
    ('Non-Fiction', 'Non-Fictional books')
]

books = [
    ('Sample Book', '2021-01-01', 19.99, 1, 1),
    ('Sci-Fi Adventure', '2022-05-15', 25.99, 2, 2),
    ('Historical Journey', '2020-08-10', 15.49, 3, 3),
    ('Fictional Drama', '2019-11-23', 9.99, 1, 1),
    ('Space Odyssey', '2023-02-20', 30.00, 2, 2),
    ('Biography of a Legend', '2021-06-14', 20.00, 3, 3),
    ('Another Fiction Book', '2020-01-01', 12.99, 1, 1),
    ('Science Facts', '2022-12-01', 18.99, 2, 3),
    ('Fantasy World', '2021-07-21', 22.50, 1, 2),
    ('Technology Advances', '2020-03-15', 17.75, 2, 3)
]

# Insert authors
cursor.executemany("INSERT INTO Authors (first_name, last_name, bio) VALUES (?, ?, ?)", authors)

# Insert genres
cursor.executemany("INSERT INTO Genres (genre_type, description) VALUES (?, ?)", genres)

# Insert books
cursor.executemany("INSERT INTO Books (title, published_date, price, author_id, genre_id) VALUES (?, ?, ?, ?, ?)", books)
conn.commit()

# Search function
def search_books(query):
    cached_result = get_from_cache(query)
    if cached_result:
        return cached_result

    cursor.execute("SELECT * FROM Books WHERE title LIKE ?", ('%' + query + '%',))
    result = cursor.fetchall()
    cache_search(query, result)
    return result

# Fetch best sellers (mock implementation)
def get_best_sellers():
    cursor.execute("SELECT * FROM Books ORDER BY book_id LIMIT 10")
    return cursor.fetchall()

# Recommendations based on previous books and wishlist (mock implementation)
user_read_books = {
    'user_1': [1, 3],  # user 1 read "Sample Book" and "Historical Journey"
    'user_2': [2, 5]   # user 2 read "Sci-Fi Adventure" and "Space Odyssey"
}

def get_recommendations(user_id):
    # Mock logic: recommend books from the same genres as previously read books
    read_books = user_read_books.get(user_id, [])
    if not read_books:
        return []

    # Get genres of books the user has read
    genre_ids = set()
    for book_id in read_books:
        cursor.execute("SELECT genre_id FROM Books WHERE book_id = ?", (book_id,))
        genre_ids.add(cursor.fetchone()[0])

    # Recommend other books from the same genres
    genre_ids = tuple(genre_ids)
    cursor.execute(f"SELECT * FROM Books WHERE genre_id IN ({','.join(['?' for _ in genre_ids])})", genre_ids)
    return cursor.fetchall()

# Example usage
create_session('user_1')
create_session('user_2')

print("Session valid for user_1:", is_session_valid('user_1'))
print("Session valid for user_2:", is_session_valid('user_2'))

time.sleep(2)  # Simulate time passing

print("Search results for 'Sci-Fi':", search_books('Sci-Fi'))
print("Best sellers:", get_best_sellers())
print("Recommendations for user_1:", get_recommendations('user_1'))
print("Recommendations for user_2:", get_recommendations('user_2'))

# Close the connection
conn.close()
